In [140]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import warnings
warnings.filterwarnings('ignore')
import pingouin as pg
import statsmodels.api as sm

In [141]:
data = pd.read_pickle('data99.pkl')
data_use = data.copy()

factors = ['MV', 'BV', 'SV', 'MG', 'BG', 'SG']
# data_use = data_use.drop(columns=[col for col in factors if (data_use[col] == 0).all()])


In [142]:
#########################
### Table 1 Panel B #####
#########################

factors = ['MV', 'BV', 'SV', 'MG', 'BG', 'SG']

for factor in factors:
    data_use[f'{factor}_me'] = data_use[factor] * data_use['me']
grouped = data_use.groupby('jdate')[[f'{factor}_me' for factor in factors]].sum()
grouped['total_me'] = data_use.groupby('jdate')['me'].sum()
for factor in factors:
    grouped[factor] = grouped[f'{factor}_me'] / grouped['total_me']
avg_market_cap_percentage = grouped[factors].mean() * 100
avg_market_cap_percentage = avg_market_cap_percentage.round(1)

table1_panelB = pd.DataFrame([avg_market_cap_percentage.values], columns=factors)
table1_panelB.index = ['Average percent of MC']
table1_panelB

## 13.9 11.6 2.3 51.9 48.4 3.5 ##

,MV,BV,SV,MG,BG,SG
Average percent of MC,14.0,11.7,2.3,51.3,47.8,3.5


In [143]:
#########################
### Table 1 Panel A #####
#########################
data_use = data.copy()

before = data_use[(data_use['jdate']>='1963-07-31') & (data_use['jdate']<='1991-06-30')]
after = data_use[(data_use['jdate']>'1991-06-30') & (data_use['jdate']<='2019-06-30')]
whole = data_use.copy()

In [144]:
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan

def calculate_portfolio_returns(data, portfolios,var,weight):
    result = pd.DataFrame()

    for portfolio in portfolios:
        subset = data[data[portfolio] == 1]
        result[portfolio] = subset.groupby('jdate').apply(wavg, var, weight)
    result['all_vwret'] = data.groupby('jdate').apply(wavg, var, weight)

    for portfolio in portfolios:
        result[portfolio + '_excess'] = result[portfolio] - result['all_vwret']

    portfolio_return = result.reset_index()
    return portfolio_return

portfolios = ['MV', 'BV', 'SV', 'MG', 'BG', 'SG']

portfolio_return = calculate_portfolio_returns(whole,portfolios,'ret_ahead','me')

In [145]:
portfolio_return.to_pickle('portfolio_return.pkl')

In [146]:
portfolio_return00 = portfolio_return.copy()
portfolio_return00['post_dummy'] = (portfolio_return00['jdate'] <= '1991-06-30').astype(int)
subset = portfolio_return00.copy()
subset00 = portfolio_return00[portfolio_return00['post_dummy']==1].reset_index(drop=True)
subset11 = portfolio_return00[portfolio_return00['post_dummy']==0].reset_index(drop=True)
subset99 = subset00 - subset11

In [147]:
def summary(data, portfolio):
    portfolio_name = portfolio + '_excess'
    _data00 = data[portfolio_name]
    _data00 = _data00

    mean = _data00.mean()  
    var = _data00.var(ddof=1)  
    sd = var ** 0.5  
    n = len(_data00)  
    se = (var / n) ** 0.5 
    t_stat_diff = mean / se 

    result = pd.DataFrame(
        [
            mean ,  
            sd ,   
            t_stat_diff  
        ],
        index=['Average Excess Return', 'Standard Deviation', 't-Statistic'],
        columns=[portfolio]
    )

    return result

In [148]:
result00 = pd.concat(
    [summary(subset, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

result11 = pd.concat(
    [summary(subset00, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

result22 = pd.concat(
    [summary(subset11, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

result33 = pd.concat(
    [summary(subset99, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

table1_panelA = pd.concat([result00,result11,result22,result33], axis=0)
table1_panelA

,MV,BV,SV,MG,BG,SG
Average Excess Return,0.24,0.17,0.47,-0.02,-0.02,0.06
Standard Deviation,2.65,2.68,3.63,1.09,1.13,3.67
t-Statistic,2.39,1.65,3.32,-0.59,-0.45,0.43
Average Excess Return,0.40,0.33,0.58,-0.04,-0.04,0.13
Standard Deviation,2.30,2.22,3.30,1.20,1.24,3.44
t-Statistic,3.15,2.69,3.24,-0.63,-0.56,0.68
Average Excess Return,0.09,0.02,0.35,-0.01,-0.00,-0.01
Standard Deviation,2.95,3.07,3.93,0.96,1.01,3.89
t-Statistic,0.57,0.09,1.62,-0.16,-0.02,-0.03
Average Excess Return,0.30,0.31,0.23,-0.03,-0.04,0.13


In [151]:
result00 = pd.concat(
    [summary(subset, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

result11 = pd.concat(
    [summary(subset00, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

result22 = pd.concat(
    [summary(subset11, portfolio=portfolio) for portfolio in portfolios], 
    axis=1
).round(2)

portfolio_return00 = portfolio_return.copy()
portfolio_return00['post_dummy'] = (portfolio_return00['jdate'] <= '1991-06-30').astype(int)

result33 = pd.DataFrame(columns=['MV', 'BV', 'SV', 'MG', 'BG', 'SG'])

portfolios = ['MV', 'BV', 'SV', 'MG', 'BG', 'SG']
for portfolio in portfolios:
    y = portfolio_return00[portfolio + '_excess']  
    X = sm.add_constant(portfolio_return00['post_dummy']) 
    model = sm.OLS(y, X).fit()

    coefficient = model.params['post_dummy']
    std_err = model.bse['post_dummy']
    std_resid = model.mse_resid ** 0.5
    t_stat = model.tvalues['post_dummy']
    result33[portfolio] = [coefficient , std_resid , t_stat]
result33.index = ['Average Excess Return', 'Standard Deviation', 't-Statistic']

table1_panelA_v2 = pd.concat([result00,result11,result22,result33], axis=0).round(2)
table1_panelA_v2

,MV,BV,SV,MG,BG,SG
Average Excess Return,0.24,0.17,0.47,-0.02,-0.02,0.06
Standard Deviation,2.65,2.68,3.63,1.09,1.13,3.67
t-Statistic,2.39,1.65,3.32,-0.59,-0.45,0.43
Average Excess Return,0.40,0.33,0.58,-0.04,-0.04,0.13
Standard Deviation,2.30,2.22,3.30,1.20,1.24,3.44
t-Statistic,3.15,2.69,3.24,-0.63,-0.56,0.68
Average Excess Return,0.09,0.02,0.35,-0.01,-0.00,-0.01
Standard Deviation,2.95,3.07,3.93,0.96,1.01,3.89
t-Statistic,0.57,0.09,1.62,-0.16,-0.02,-0.03
Average Excess Return,0.30,0.31,0.23,-0.03,-0.04,0.13


In [152]:
portfolio_return11 = portfolio_return.copy()

In [153]:
#########################
### Table 1 Panel C #####
#########################

variables = ['all_vwret', 'MV', 'BV', 'SV', 'MG', 'BG', 'SG']
subset = portfolio_return11[variables] 
corr_matrix = subset.corr() 
corr_matrix00 = pd.DataFrame(corr_matrix.iloc[0, 1:].round(2)).T 

variables = ['MV_excess', 'BV_excess', 'SV_excess', 'MG_excess', 'BG_excess', 'SG_excess']
subset = portfolio_return11[variables]
corr_matrix11 = subset.corr().round(2)  
corr_matrix11.columns = corr_matrix11.columns.str.replace('_excess', '')  
corr_matrix11.index = corr_matrix11.index.str.replace('_excess', '') 

corr_matrix = pd.concat([corr_matrix00, corr_matrix11])
corr_matrix.index.values[0] = 'Cor(R, RM)' 
table1_panelC = corr_matrix

In [154]:
#########################
###  Hotelling T^2  #####
#########################
X = subset00[variables]
Y = subset11[variables]
print(pg.multivariate_ttest(X, Y,paired=False))

                 T2         F  df1  df2      pval
hotelling  4.778561  0.790483    6  665  0.577532


In [155]:
print(table1_panelA)
print(pg.multivariate_ttest(X, Y,paired=False))
print(table1_panelB)
print(table1_panelC)

                         MV    BV    SV    MG    BG    SG
Average Excess Return  0.24  0.17  0.47 -0.02 -0.02  0.06
Standard Deviation     2.65  2.68  3.63  1.09  1.13  3.67
t-Statistic            2.39  1.65  3.32 -0.59 -0.45  0.43
Average Excess Return  0.40  0.33  0.58 -0.04 -0.04  0.13
Standard Deviation     2.30  2.22  3.30  1.20  1.24  3.44
t-Statistic            3.15  2.69  3.24 -0.63 -0.56  0.68
Average Excess Return  0.09  0.02  0.35 -0.01 -0.00 -0.01
Standard Deviation     2.95  3.07  3.93  0.96  1.01  3.89
t-Statistic            0.57  0.09  1.62 -0.16 -0.02 -0.03
Average Excess Return  0.30  0.31  0.23 -0.03 -0.04  0.13
Standard Deviation     3.63  3.64  4.96  1.55  1.62  5.03
t-Statistic            1.53  1.56  0.86 -0.39 -0.41  0.49
                 T2         F  df1  df2      pval
hotelling  4.778561  0.790483    6  665  0.577532
                         MV    BV   SV    MG    BG   SG
Average percent of MC  14.0  11.7  2.3  51.3  47.8  3.5
              MV    BV    SV    MG

In [109]:
table1_panelA.to_excel("/Users/fuhk/Downloads/table_panelA.xlsx")